In [1]:
import sys
sys.path.append('../src')

In [2]:
import numpy as np

In [3]:
x = np.array([1,1,1,2,2,2,5,25,1,1])

## Min

In [4]:
def calculate_min(signal):
    return np.min(signal)

In [5]:
print(calculate_min(x))

1


## Max

In [6]:
def calculate_max(signal):
    return np.max(signal)

In [7]:
print(calculate_max(x))

25


## Energy

$$
Energy = \sum_{x=x_0}^{x_0+N} y_x^2
$$


In [8]:
def calculate_energy(signal):
    return np.sum(np.power(signal,2))

In [9]:
print(calculate_energy(x))

667


## Line Length
$$
LineLength = \sum_{x=x_0+1}^{x_0+N}|y_x - y_{x-1}|
$$

In [10]:
def calculate_line_length(signal):
    offset_signal = signal[1:].copy()
    return(np.sum(np.abs(offset_signal-signal[:-1])))

In [11]:
print(calculate_line_length(x))

48


## Moving average

$$
MovingAverage = \frac{1}{N}\sum_{x=x_0}^{x_0+N}y_x
$$


In [12]:
def calculate_moving_avg(signal):
    return np.mean(signal)

In [13]:
print(calculate_moving_avg(x))

4.1


## Standard deviation
$$
StandardDeviation = \frac{1}{N}\sum_{x=x_0}^{x_0+N}(y_x-\mu)^2
$$


In [14]:
def calculate_std(signal):
    return np.std(signal)

In [15]:
print(calculate_std(x))

7.063285354564121


## Skewness

$$Skewness = \frac{\sqrt{N(N-1)}}{N(N-2)} g$$
$g = \frac{\sum_{x=x_0}^{x_0+N} (y_x - \mu)^3}{\sigma^3}$

In [16]:
def calculate_skewness(signal):
    std = np.std(signal)
    mean = np.mean(signal)
    n = len(signal)
    coeff = (n*(n-1))**0.5/(n*(n-2))
    skewness = np.sum(np.power(signal-mean,3))/std**3
    
    return coeff * skewness

In [17]:
print(calculate_skewness(x))

3.0129721592204315


## Kurtosis

$$Kurtosis = \frac{N-1}{(N-2)(N-3)}[(N+1) g + 6]$$

$g = \frac{\sum_{x=x_0}^{x_0+N} (y_x - \mu)^4}{\sigma^2} - 3$

In [18]:
def calculate_kurtosis(signal):
    std = np.std(signal)
    mean = np.mean(signal)
    n = len(signal)
    coeff = (n-1)/((n-2)*(n-3))
    kurtosis = np.sum(np.power(signal-mean,4))/std**2
    
    return coeff*((n+1)*kurtosis + 6)

In [19]:
print(calculate_kurtosis(x))

6780.540053474785


## Shannon entropy

$$ShannonEntropy = \sum_{x=x_0}^{x_0+N} \text{freq}(y_x) \text{log}(\text{freq}(y_x))  $$

In [20]:
def calculate_shannon_entropy(signal):
    unique, counts = np.unique(signal, return_counts=True)
    occurences = dict(zip(unique, counts))
    n = len(unique)
    frequencies = np.array([occurences[i] for i in signal])/n
    return np.sum(frequencies*np.log(frequencies))

In [21]:
print(calculate_shannon_entropy(x))

0.05421535213735873
